## 2018-08-29
- 크롤링 방식 선정 관련
>- 다음 웹툰의 경우 json 형태로 데이터를 읽어오면 손쉽게 코멘트와 부수정보를 얻을 수 있음.
    - 메인화면
        - 특정 만화 목록 [기다리면무료 json ex)] : http://webtoon.daum.net/data/pc/webtoon/list_gidamoo
        - 특정 만화 목록 [완결 json ex)] : http://webtoon.daum.net/data/pc/webtoon/list_finished/?genre_id=&timeStamp=1535525278698
        - 얻을 수 있는 정보 목록 :
            - 만화 목록
            - 만화의 간략정보
    - 만화 정보 및 회차 url : http://webtoon.daum.net/webtoon/view/AnOutcast
        - 만화 정보 및 회차목록 json ex) : http://webtoon.daum.net/data/pc/webtoon/view/AnOutcast?timeStamp=1535525697456
        - 얻을 수 있는 정보 목록 :
            - 장르카테고리 : 확인된건 한번에 5개씩(??)
            - 만화의 전체 회차
            - 글/작가 artist정보
            - 만화의 제목 및 닉네임
            - 평점 score
            - 추천되는 다른 만화의 정보
    - 만화 내용 및 댓글 url : http://webtoon.daum.net/webtoon/viewer/54701
        - 만화 회차 댓글목록 json ex) : http://hestia.webtoon.daum.net/hestia/api/v1/comment/webtoon_ep/54701?page=1&pageSize=99999&sortType=recommend&1535524756659
        - 얻을 수 있는 정보 목록 : 
            - 볼수 있는 만화회차의 댓글전체, 추천/비추천 현황(유료제외)

In [18]:
# scrapy

In [3]:
import scrapy
import requests
from scrapy.http import TextResponse

In [12]:
req = requests.get('http://webtoon.daum.net/#sort=free&genre=&tab=finish')
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [17]:
response.xpath('//*[@id="daumServiceLogo"]')

[<Selector xpath='//*[@id="daumServiceLogo"]' data='<a href="/" id="daumServiceLogo">\n\t\t\t\t\t<'>]

In [19]:
# bs4-json

In [22]:
# 네이버주식정보 가져오기
import pandas as pd
from pandas.io.json import json_normalize

    # url제조함수
def make_url(pageSize=10, page=1):
    return 'http://webtoon.daum.net/data/pc/webtoon/list_finished/?genre_id=&timeStamp=1535525278698'

    # 데이터를 가져와서 판다스의 데이터프레임으로 만드는 함수
def get_data(url):
    response = requests.get(url)
    json_info = response.json() #json -> dict 형식으로 알아서 바꿔줌
    a = json_info
#     companys = json_info["result"]["itemList"] #list 형태로 바꿔줌
#     a = json_normalize(companys)
    
    #행이름 바꾸기
#     a.columns = ["주식수량","거래량","종목코드","등락율","전일비","시가총액","mt","종목","시세","전일가","rf"]

    # 일일이 열에 넣는방법은 아래와 같다    
#     df = pd.DataFrame(columns=["종목","시세","전일비","등락율","시가총액","거래량"])
    
#     for company in companys:
#         df.loc[len(df)] = {
#             "종목":company["nm"],
#             "시세":company["nv"],
#             "전일비":company["cv"],
#             "등락율":company["cr"],
#             "시가총액":company["mks"],
#             "거래량":company["aq"]
#         }
    
    return a
    
    
#---함수를 써보자--------------------------    
url = make_url(5,1)
datas = get_data(url)
datas

{'result': {'status': '200', 'message': ''},
 'data': [{'id': 1518,
   'nickname': 'animalhero',
   'webtoonType': 'series',
   'title': '애니멀 히어로-닥터 슈바이처',
   'finishYn': 'Y',
   'thumbnailImage2': {'id': 0,
    'url': 'http://t1.daumcdn.net/webtoon/op/d1776151dc67c4cba16dd8c94f8c3b61f398f4a9'},
   'pcThumbnailImage': {'id': 0,
    'url': 'http://t1.daumcdn.net/webtoon/op/c49acc62577c282aa55fb3470d3e3824ba7499c7'},
   'finishDate': '20180829000000',
   'ageGrade': 0,
   'dateCreated': '20180730160439',
   'cartoon': {'artists': [{'name': '유쏘', 'penName': '유쏘'},
     {'name': '백준', 'penName': '백준'}]},
   'latestWebtoonEpisode': {'id': 55088,
    'episode': 5,
    'title': '5화 호더(hoarder) - 목욕탕의 여신',
    'thumbnailImage': {'id': 55088,
     'url': 'http://t1.daumcdn.net/webtoon/op/6b4e20e8dd5cc8a7ca4c42368261a4e6ca8e47fb'},
    'encryptUseYn': 'N',
    'serviceStatus': 'service',
    'articleId': 0,
    'dateCreated': '20180828114014',
    'price': 0,
    'isTopRecommend': False,
    'is

In [40]:
datas['data'][0].keys()

dict_keys(['id', 'nickname', 'webtoonType', 'title', 'finishYn', 'thumbnailImage2', 'pcThumbnailImage', 'finishDate', 'ageGrade', 'dateCreated', 'cartoon', 'latestWebtoonEpisode', 'webtoonServices', 'isNew', 'averageScore', 'gidamoo'])

In [33]:
ls_titles = []
ls_nicknames = []
for i in datas['data']:
    ls_titles.append(i['title'])
    ls_nicknames.append(i['nickname'])
ls_titles[:5], ls_nicknames[:5]

(['애니멀 히어로-닥터 슈바이처', '너는 아름답다', '강철강', '골든왈츠', '위 아 애니멀스'],
 ['animalhero', 'yourbeautiful', 'steelsteel', 'GoldenWaltz', 'WeAreAnimals'])

In [42]:
url2 = 'http://webtoon.daum.net/data/pc/webtoon/view/animalhero'

datas2 = get_data(url2)
datas2

{'result': {'status': '200', 'message': ''},
 'page': None,
 'data': {'artistComment': None,
  'anotherWebtoons': [{'id': 1337,
    'nickname': 'WeAreAnimals',
    'webtoonType': 'series',
    'title': '위 아 애니멀스',
    'englishTitle': None,
    'finishYn': 'Y',
    'titleImage2': {'id': 0,
     'url': 'http://t1.daumcdn.net/cartoon/589BF64D022CB10001',
     'name': None,
     'size': None,
     'width': 0,
     'height': 0,
     'mediaType': None,
     'encryptKey': None,
     'serviceStatus': 'service'},
    'thumbnailImage': {'id': 0,
     'url': 'http://t1.daumcdn.net/cartoon/589BF60C01037D0001',
     'name': None,
     'size': None,
     'width': 0,
     'height': 0,
     'mediaType': None,
     'encryptKey': None,
     'serviceStatus': 'service'},
    'thumbnailImage2': {'id': 0,
     'url': 'http://t1.daumcdn.net/cartoon/58DCAA9F060A350001',
     'name': None,
     'size': None,
     'width': 0,
     'height': 0,
     'mediaType': None,
     'encryptKey': None,
     'serviceStatus

In [43]:
datas2.keys()

dict_keys(['result', 'page', 'data', 'metaData'])

In [67]:
datas2['data']['webtoon']

{'id': 1518,
 'nickname': 'animalhero',
 'webtoonType': 'series',
 'title': '애니멀 히어로-닥터 슈바이처',
 'englishTitle': None,
 'finishYn': 'Y',
 'titleImage2': None,
 'thumbnailImage': None,
 'thumbnailImage2': {'id': 0,
  'url': 'http://t1.daumcdn.net/webtoon/op/d1776151dc67c4cba16dd8c94f8c3b61f398f4a9',
  'name': None,
  'size': None,
  'width': 0,
  'height': 0,
  'mediaType': None,
  'encryptKey': None,
  'serviceStatus': 'service'},
 'padThumbnailImage': None,
 'artistCommentImage': None,
 'homeThumbnailImage': None,
 'appThumbnailImage': {'id': 0,
  'url': 'http://t1.daumcdn.net/webtoon/op/2ad9fd43ace58f8b462628687b6b5a874b1ea29b',
  'name': None,
  'size': None,
  'width': 0,
  'height': 0,
  'mediaType': None,
  'encryptKey': None,
  'serviceStatus': 'service'},
 'pcThumbnailImage': {'id': 0,
  'url': 'http://t1.daumcdn.net/webtoon/op/c49acc62577c282aa55fb3470d3e3824ba7499c7',
  'name': None,
  'size': None,
  'width': 0,
  'height': 0,
  'mediaType': None,
  'encryptKey': None,
  'ser